In [1]:
import os
os.environ["PYOPENGL_PLATFORM"] = "egl" #opengl seems to only work with TPU

import time
from absl import flags, app
import sys
sys.path.insert(0,'third_party')
import numpy as np
import torch
import torchvision
import glob
import pdb
import cv2
import trimesh
from scipy.spatial.transform import Rotation as R
import imageio
import pyrender
from nnutils.train_utils import v2s_trainer
from nnutils.geom_utils import extract_mesh_simp, zero_to_rest_bone, \
                                zero_to_rest_dpose, skinning, lbs, se3exp_to_vec
from utils.io import save_vid, str_to_frame, save_bones, bones_to_mesh
from explore import get_center_crop, render_mesh

opts = flags.FLAGS

Loaded compiled 3D CUDA chamfer distance


In [2]:
logdir='logdir/dog80-v0/'
seqname='dog80'

In [3]:
# load flags
args = opts.read_flags_from_files(['--flagfile=%s/opts.log'%logdir, 
                                   '--nolineloade',
                                   'seqname=%s'%seqname, 
                                   '--full_mesh', 
                                   '--noce_color', 
                                   ])
unknown_flags, unparsed_args = opts._parse_args(args, known_only=True)
opts.mark_as_parsed()
opts.validate_all_flags()

In [4]:
from IPython.utils import io
with io.capture_output() as captured:
    # to avoid output oflow out of the box
    if opts.logname not in opts.model_path:
        opts.model_path = 'logdir/%s/params_latest.pth'%opts.logname
    os.mkdir('tmp') if not os.path.exists('tmp') else None
    trainer = v2s_trainer(opts, is_eval=True)
    data_info = trainer.init_dataset()
    trainer.define_model(data_info)
    seqname=opts.seqname

    model = trainer.model
    model.eval()

In [5]:
# params
img_size = 256
grid_size= 64

show_rest_pose = False
show_deform = False
apply_lbs = True
mlp_deform = False


cam_offset = np.asarray([0,0,0.6])

focal_fac = 3

renderer = pyrender.OffscreenRenderer(img_size, img_size)

In [6]:
import matplotlib
fid = torch.Tensor(range(0,len(model.impath))).cuda().long()
D=model.pose_code(fid)
D = D.view(len(fid),-1)
Dmean=D.mean(0)[None]

D = D-Dmean
A = D.T.matmul(D)/D.shape[0] # fxf
U,S,V=torch.svd(A) #
code_proj_3d=D.matmul(V[:,:3])
time = np.asarray(range(len(model.impath)))
time = time/time.max()
code_proj_3d=code_proj_3d.detach().cpu().numpy()

In [7]:
def plot(vidid, frameid, angle, xcrd, ycrd):
    with torch.no_grad():
        # randomly sample code beta
        embedid=frameid
        canonical_rot = cv2.Rodrigues(np.asarray([0,np.pi/3-np.pi*angle/180,0]))[0]

        # get target image
        img_path = data_info['impath'][embedid]
        target_img = get_center_crop(img_path, img_size=img_size//4)

        # get mesh
        #vidid=14
        beta_code = model.nerf_coarse.vid_code.weight
        mesh_rest = extract_mesh_simp(model, opts.chunk, \
                    grid_size, 0, vidid=vidid, is_eval=False, beta=beta_code[vidid])
        num_pts = mesh_rest.vertices.shape[0]

        # stretch + lbs: mesh to mesh
        query_vidid = vidid*torch.ones(1).long().to(model.device)
        pts_can=torch.Tensor(mesh_rest.vertices).to(model.device)
        pts_can = pts_can[:,None]
        query_time = torch.ones(1,1).long().to(model.device)*embedid

        #jlen_scale = model.nerf_body_rts.compute_jlen_scale(query_vidid)
        jlen_scale = model.nerf_body_rts.jlen_scale_z + model.nerf_body_rts.jlen_scale[vidid]
        sim3=model.nerf_body_rts.sim3+model.nerf_body_rts.sim3_vid[vidid]
        sim3[:7] = model.nerf_body_rts.sim3[:7]
        sim3=sim3[None]
        
        # zero-to-rest
        bones_rst, bone_rts_rst = zero_to_rest_bone(model, model.bones)

        # stretch + fk
        #show_rest_pose=True #TODO
        if xcrd==0 and ycrd==0:
            if show_rest_pose:
                query_code = torch.zeros(1,opts.t_embed_dim).to(model.device)
            else:
                query_code = model.nerf_body_rts.pose_code(query_time)
        else:
            query_code = torch.Tensor([xcrd,ycrd]).to(model.device)
            query_code = query_code[None] @ V[:,:2].T + Dmean
        
        bone_rts_fw = model.nerf_body_rts.forward_decode(query_code,
                                query_vidid, show_rest_pose=show_rest_pose,
            jlen_scale_in=jlen_scale, sim3_in=sim3)
        bone_rts_fw = zero_to_rest_dpose(opts, bone_rts_fw, bone_rts_rst)
        bone_rts_fw = bone_rts_fw.repeat(num_pts, 1,1)

        # skin
        rest_pose_code = model.rest_pose_code(torch.Tensor([0]).long().to(model.device))
        skin_forward,_ = skinning(pts_can, model.embedding_xyz, bones_rst,
                            rest_pose_code, model.nerf_skin)

        # dqs
        pts_dfm,bones_dfm = lbs(bones_rst, bone_rts_fw, skin_forward,
                pts_can,backward=False)

        # re-create mesh
        mesh_dfm = trimesh.Trimesh( pts_dfm[:,0].cpu().numpy(), mesh_rest.faces,
                                vertex_colors=mesh_rest.visual.vertex_colors)
        
        color = render_mesh(renderer, mesh_dfm, canonical_rot, cam_offset, focal_fac, img_size)

        # get joint vis
        joints,_ = model.nerf_body_rts.forward_abs(x=query_time,vid=query_vidid,
                show_rest_pose=show_rest_pose,
                jlen_scale_in=jlen_scale, sim3_in=sim3)
        joints = joints.view(1,-1,12)
        joints = torch.cat([joints[:,:,:9].view(1,-1,3,3),
                            joints[:,:,9:].view(1,-1,3,1)],-1)
        joints = se3exp_to_vec(joints[0]).cpu().numpy()
        joint_mesh = bones_to_mesh(joints, 0.1, parent=model.robot.urdf.parent_idx)

        j_col = render_mesh(renderer, joint_mesh, canonical_rot, cam_offset, focal_fac, img_size)

    plt.plot(code_proj_3d[:,0], code_proj_3d[:,1], marker='.', color='g', linestyle='None')
    plt.plot(xcrd, ycrd, color='r', marker='x')

    fig = plt.gcf()
    fig.canvas.draw()
    plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    plot = plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plot = cv2.resize(plot, (img_size,img_size))

    images = [target_img/255, color/255, j_col/255, plot/255]
    #images = [target_img/255, color/255]
    
    mediapy.show_images(images)

In [8]:
import ipywidgets as widgets
from ipywidgets import GridspecLayout
import mediapy
%matplotlib inline
from matplotlib import pyplot as plt
plt.ioff()
import matplotlib
cmap = matplotlib.cm.get_cmap('cool')

vidid = widgets.IntSlider(min=0,max=model.num_vid,step=1, description="Video id")
frameid = widgets.IntSlider(min=0,max=model.num_fr,step=1, description="Frame id")
angle = widgets.IntSlider(min=0,max=360, step=1, description="Angle")
xcrd = widgets.FloatSlider(min=-3,max=3,step=0.1, description="X-coord")
ycrd = widgets.FloatSlider(min=-3,max=3, step=0.1, description="Y-coord")

# layout
ui = GridspecLayout(3, 2)
ui[0,0]=vidid
ui[1,0]=frameid
ui[2,0]=angle
ui[0,1]=xcrd
ui[1,1]=ycrd
ui.width='%dpx'%(img_size*3)

out = widgets.interactive_output(plot, {'vidid':vidid, 'frameid': frameid, 'angle': angle, 'xcrd': xcrd, 'ycrd': ycrd})
out.layout.height = '%dpx'%(img_size*1.5)

display(out, ui)

Output(layout=Layout(height='384px'))

GridspecLayout(children=(IntSlider(value=0, description='Video id', layout=Layout(grid_area='widget001'), max=…